In [1]:
import requests
import pandas as pd

#显示所有列
pd.set_option('display.max_columns', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [2]:
def read_lives_file(file:str):
    with open(file,encoding='utf8') as lives_file:
        lives=lives_file.readlines()
    
    return lives

In [3]:
def requests_live(url:str):
    try:
        response_url=requests.get(url,timeout=5)
        if response_url.status_code==200:
            return True
        else:
            return False
    except:
        return False

# 解析m3u文件，并验证频道可用性

In [4]:
def parse_line_group_title(line1):
    line1_splits=line1.split(',')[0].split()
    for line1_split in line1_splits:
        if 'group-title' in line1_split:
            group_title=line1_split.split('=')[1].replace('"','').replace("'",'')
            
            return group_title

    return ''

In [14]:
def parse_line(line1,line2):
    line1=line1.strip()
    line2=line2.strip()
    
    channel_group=channel_name=channel_url=channel_type=''
    
    if line1.startswith("#EXTINF:") and (line2.startswith("http://[") or line2.startswith("https://[")):
        channel_group=parse_line_group_title(line1)
        channel_name=line1.split(',')[1]
        channel_url=line2
        channel_type='IPV6'
    elif line1.startswith("#EXTINF:") and line2.startswith("http"):
        channel_group=parse_line_group_title(line1)
        channel_name=line1.split(',')[1]
        channel_url=line2
        channel_type='IPV4'
        
        
    return channel_group,channel_name,channel_url,channel_type

In [15]:
def parse_file_m3u(file):
    lives_file=read_lives_file(file)
    length=len(lives_file)

    cn=0
    result=[]
    while cn<length-1:
        channel_group,channel_name,channel_url,channel_type=parse_line(lives_file[cn],lives_file[cn+1])
        print(lives_file[cn],lives_file[cn+1])
        print('channel_group:',channel_group)
        print('channel_name:',channel_name)
        print('channel_url:',channel_url)
        print('channel_type:',channel_type)
        print('----------------------')
        if channel_url:
            #is_available=requests_live(channel_url)
            is_available=0
            cn+=2
        else:
            is_available=''
            cn+=1
            
        if channel_url:
            result.append([channel_group,channel_name,channel_url,channel_type,is_available])

        
        
    return result

In [24]:
result=parse_file_m3u('./收集的直播源/CCTV.m3u8')

﻿#EXTM3U
 #EXTINF:-1 ,CCTV4K-25P-HLG源

channel_group: 
channel_name: 
channel_url: 
channel_type: 
----------------------
#EXTINF:-1 ,CCTV4K-25P-HLG源
 http://159.75.85.63:5679/cctv4k.php

channel_group: 
channel_name: CCTV4K-25P-HLG源
channel_url: http://159.75.85.63:5679/cctv4k.php
channel_type: IPV4
----------------------
#EXTINF:-1 ,CCTV4K-50P-HLG小果源
 http://159.75.85.63:5681/sxg.php?id=ys4Kcq_2000

channel_group: 
channel_name: CCTV4K-50P-HLG小果源
channel_url: http://159.75.85.63:5681/sxg.php?id=ys4Kcq_2000
channel_type: IPV4
----------------------
#EXTINF:-1 ,CCTV16-4K-50P-HLG小果源
 http://159.75.85.63:5681/sxg.php?id=CCTV-16-4K_8000

channel_group: 
channel_name: CCTV16-4K-50P-HLG小果源
channel_url: http://159.75.85.63:5681/sxg.php?id=CCTV-16-4K_8000
channel_type: IPV4
----------------------
#EXTINF:-1 ,CCTV9
 http://222.240.82.92:9901/tsfile/live/0009_1.m3u8

channel_group: 
channel_name: CCTV9
channel_url: http://222.240.82.92:9901/tsfile/live/0009_1.m3u8
channel_type: IPV4
-----------

In [25]:
data=pd.DataFrame(result,columns=['频道组','频道名称','频道地址','频道类型','是否可用'])

In [26]:
data.head()

,频道组,频道名称,频道地址,频道类型,是否可用
0,,CCTV4K-25P-HLG源,http://159.75.85.63:5679/cctv4k.php,IPV4,0
1,,CCTV4K-50P-HLG小果源,http://159.75.85.63:5681/sxg.php?id=ys4Kcq_2000,IPV4,0
2,,CCTV16-4K-50P-HLG小果源,http://159.75.85.63:5681/sxg.php?id=CCTV-16-4K_8000,IPV4,0
3,,CCTV9,http://222.240.82.92:9901/tsfile/live/0009_1.m3u8,IPV4,0
4,,CCTV8,http://222.240.82.92:9901/tsfile/live/0008_1.m3u8,IPV4,0


In [27]:
writer = pd.ExcelWriter('直播-M3U-3.xlsx', engine='xlsxwriter',
                            engine_kwargs={'options': {'strings_to_urls': False}})
data.to_excel(writer, index=False)
writer.close()

# 解析TxT文件，并验证频道可用性

In [5]:
def parse_file_txt(file):
    lives_file=read_lives_file(file)
    length=len(lives_file)

    result=[]
    for channel in lives_file:
        if channel.strip():
            channel_name,channel_url=channel.strip().split(',',1)
            
            #is_available=requests_live(channel_url)
            is_available=0
            
            if channel_url.startswith("http://[") or channel_url.startswith("https://["):
                channel_type='IPV6'
            else:
                channel_type='IPV4'
            
            
            print('channel_name:',channel_name)
            print('channel_url:',channel_url)
            print('channel_type:',channel_type)
            print('----------------------')
            
            result.append([channel_name,channel_url,channel_type,is_available])
        
        
    return result

In [10]:
result=parse_file_txt('./收集的直播源/11.txt')

channel_name: 🔥热辣滚烫TC
channel_url: https://ikcdn01.ikzybf.com/20240212/XsOkP0kD/2000kb/hls/index.m3u8
channel_type: IPV4
----------------------
channel_name: 🔥飞驰人生2TC
channel_url: https://ikcdn01.ikzybf.com/20240212/jjCpFTXC/2000kb/hls/index.m3u8
channel_type: IPV4
----------------------
channel_name: 🔥金手指[国]
channel_url: https://ikcdn01.ikzybf.com/20240213/xZTh2qQA/2000kb/hls/index.m3u8
channel_type: IPV4
----------------------
channel_name: 🔥一闪一闪亮星星
channel_url: https://ikcdn01.ikzybf.com/20240213/vWZ9ePUu/2000kb/hls/index.m3u8
channel_type: IPV4
----------------------
channel_name: 志愿军：雄兵出击
channel_url: https://dghls.mcloud.139.com/hls/L1c36c04a53255f72579c1f62fe2ad8e1b/single/video/0/376/index.m3u8
channel_type: IPV4
----------------------
channel_name: 我爸没说的那件事
channel_url: https://dghls.mcloud.139.com/hls/LF68795969fea240236a03c69349af2fbd/single/video/0/480/index.m3u8
channel_type: IPV4
----------------------
channel_name: 斗破苍穹·止戈
channel_url: https://dghls.mcloud.139.com/hls/LP

channel_type: IPV4
----------------------
channel_name: 「河北」邯郸科技教育
channel_url: https://jwcdnqx.hebyun.com.cn/live/hdkj/1500k/tzwj_video.m3u8?zhebd
channel_type: IPV4
----------------------
channel_name: 「河北」鹿泉一套
channel_url: https://jwliveqxzb.hebyun.com.cn/luquanyi/luquanyi.m3u8?zhebd
channel_type: IPV4
----------------------
channel_name: 「河北」鹿泉一套
channel_url: https://jwcdnqx.hebyun.com.cn/live/luquanyi/1500k/tzwj_video.m3u8?zhebd
channel_type: IPV4
----------------------
channel_name: 「河北」石家庄都市
channel_url: http://pluslive1.sjzntv.cn/dushi/playlist.m3u8?zhebd
channel_type: IPV4
----------------------
channel_name: 「河北」石家庄生活
channel_url: http://pluslive1.sjzntv.cn/shenghuo/playlist.m3u8?zhebd
channel_type: IPV4
----------------------
channel_name: 「河北」石家庄新闻综合
channel_url: http://pluslive1.sjzntv.cn/xmzh/playlist.m3u8?zhebd
channel_type: IPV4
----------------------
channel_name: 「河北」石家庄娱乐
channel_url: http://pluslive1.sjzntv.cn/yule/playlist.m3u8?zhebd
channel_type: IPV4
------------

channel_name: 阳春综合
channel_url: http://ci.emdoor.tech:8080/touchtv.php?pk=2458
channel_type: IPV4
----------------------
channel_name: 广宁综合
channel_url: http://ci.emdoor.tech:8080/touchtv.php?pk=2414
channel_type: IPV4
----------------------
channel_name: 惠东综合频道
channel_url: http://ci.emdoor.tech:8080/touchtv.php?pk=2404
channel_type: IPV4
----------------------
channel_name: 乐昌电视台
channel_url: http://ci.emdoor.tech:8080/touchtv.php?pk=2503
channel_type: IPV4
----------------------
channel_name: 惠阳电视台
channel_url: http://ci.emdoor.tech:8080/touchtv.php?pk=2470
channel_type: IPV4
----------------------
channel_name: 开平综合
channel_url: http://ci.emdoor.tech:8080/touchtv.php?pk=2405
channel_type: IPV4
----------------------
channel_name: 廉江台
channel_url: http://ci.emdoor.tech:8080/touchtv.php?pk=2445
channel_type: IPV4
----------------------
channel_name: 遂溪台
channel_url: http://ci.emdoor.tech:8080/touchtv.php?pk=2488
channel_type: IPV4
----------------------
channel_name: 吴川综合
channel_url

channel_url: https://txmov2.a.yximgs.com/mediacloud/acfun/acfun_video/8983d8c926edf403-e8bfe3f01cfa00a46eca126706bd7b1f-hls_1080p_h264_60_1.m3u8?pkey=ABBRbnXG8ub6M73LmC58Re5OToo6A8Ri64I86UPefcv6zytj5GaOBBp3auY1VHFouDRJDOfdcxTYd4Bhj8JAXIandAsR8MtUE6DC_7sLWZiknVnPOLLfTu1EAnm63boreF3g1S8IOU7iMlJArIveLlkEk9853uCGrvwqh2RmZgTwlu2Yy5M91OBRyQXXhSIBkkDANqVrKoTRsIW8Dt7P8eFS5TIWU8_nOEmGccAUJ9ky3ncc9WxMz9L9jpZamYmLfN0&safety_id=AALufSf0OHCmkfb0zHi92cpK
channel_type: IPV4
----------------------
channel_name: 泰之战①
channel_url: https://txmov2.a.yximgs.com/bs3/video-hls/5232338420747548479_hlsb.m3u8?pkey=AAUR7Wp4cKw3MEaOyv0EwH0kj1gOlii1ZrT8SUp9YTncOWbLEHJbYU-x-Yhtke7HqUQ6yr6Pbju4qJrcU5aaMmCaVrtTQFSkIPT20ZtqAdq3HfUJDPL1s0y8u3JSO6prRNg
channel_type: IPV4
----------------------
channel_name: 泰之战③
channel_url: https://txmov2.a.yximgs.com/mediacloud/acfun/acfun_video/34378b42e3e24ee1-3df5d1ca280a33eaccd2770be028f6a8-hls_720p_2.m3u8?pkey=ABCDYFJSTiiZ1tDpFhypQIxzokOgBCEhbIo5T_S26NboLDyOFciN1mccYNCQQce6SliNQ-

In [11]:
data=pd.DataFrame(result,columns=['频道名称','频道地址','频道类型','是否可用'])

In [12]:
data.head()

,频道名称,频道地址,频道类型,是否可用
0,🔥热辣滚烫TC,https://ikcdn01.ikzybf.com/20240212/XsOkP0kD/2000kb/hls/index.m3u8,IPV4,0
1,🔥飞驰人生2TC,https://ikcdn01.ikzybf.com/20240212/jjCpFTXC/2000kb/hls/index.m3u8,IPV4,0
2,🔥金手指[国],https://ikcdn01.ikzybf.com/20240213/xZTh2qQA/2000kb/hls/index.m3u8,IPV4,0
3,🔥一闪一闪亮星星,https://ikcdn01.ikzybf.com/20240213/vWZ9ePUu/2000kb/hls/index.m3u8,IPV4,0
4,志愿军：雄兵出击,https://dghls.mcloud.139.com/hls/L1c36c04a53255f72579c1f62fe2ad8e1b/single/video/0/376/index.m3u8,IPV4,0


In [13]:
writer = pd.ExcelWriter('直播-txt-2.xlsx', engine='xlsxwriter',
                            engine_kwargs={'options': {'strings_to_urls': False}})
data.to_excel(writer, index=False)
writer.close()

# 根据Excel文件生m3u格式文件

In [1]:
import pandas as pd

In [2]:
data=pd.read_excel('直播.xlsx',sheet_name='频道源',dtype='str',na_filter='')

In [3]:
with open('movie_live.m3u','w',encoding='utf8') as file:
    file.write('#EXTM3U\n\n')
    
    channel_group='央视频道'
    for _,channel in data.iterrows():
        if channel_group != channel["频道组"]:
            file.write('\n\n')
            channel_group=channel["频道组"]
            
        file.write(f'#EXTINF:-1 group-title="{channel["频道组"]}",{channel["频道名称"]}\n')
        
        file.write(f'{channel["频道地址"]}\n')
    

# 根据Excel文件生成txt格式文件

In [4]:
with open('movie_live.txt','w',encoding='utf8') as file:    
    channel_group='央视频道'
    file.write(f'央视频道,#genre#\n')
    for _,channel in data.iterrows():
        if channel_group != channel["频道组"]:
            file.write('\n\n')
            channel_group=channel["频道组"]
            file.write(f'{channel["频道组"]},#genre#\n')
        
        file.write(f'{channel["频道名称"]},{channel["频道地址"]}\n')

# 生成直播源

In [2]:
import pandas as pd

In [4]:
data = pd.read_excel('result_clean_verify.xlsx')

In [5]:
len(data)

21167

In [6]:
data_filter=data[data['地址是否可用']==1]

In [7]:
len(data_filter)

1660

In [8]:
data_filter.dtypes

来源          object
频道组         object
频道名称        object
频道地址        object
频道类型        object
清洗频道名称      object
清洗频道组名称     object
频道组排序      float64
频道排序       float64
地址是否可用       int64
测试速度       float64
dtype: object

In [9]:
data_filter_sort=data_filter.sort_values(
    by=['频道组排序','频道排序','测试速度','频道类型'],
    ascending=[True, True, False, True]
)

In [10]:
data_filter_sort.head(50)

,来源,频道组,频道名称,频道地址,频道类型,清洗频道名称,清洗频道组名称,频道组排序,频道排序,地址是否可用,测试速度
4224,https://raw.githubusercontent.com/alantang1977/iptv_api/refs/heads/main/output/live_ipv4.m3u,🥝┃央视频道,CCTV1,http://112.27.235.94:8000/hls/1/index.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,8915.056559
5,https://raw.githubusercontent.com/Guovin/iptv-api/gd/output/result.m3u,📺央视频道,CCTV1,http://[2409:8087:1:20:20::29]/otttv.bj.chinamobile.com/PLTV/88888888/224/3221226895/1.m3u8?serv...,IPV6,CCTV-1,央视频道,1.0,1.0,1,4183.204988
1,https://raw.githubusercontent.com/Guovin/iptv-api/gd/output/result.m3u,📺央视频道,CCTV1,http://php.jdshipin.com/TVOD/iptv.php?id=rthk33,IPV4,CCTV-1,央视频道,1.0,1.0,1,2882.702624
2,https://raw.githubusercontent.com/Guovin/iptv-api/gd/output/result.m3u,📺央视频道,CCTV1,http://iptv.huuc.edu.cn/hls/cctv1hd.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,1514.715191
6293,https://raw.githubusercontent.com/jiangnan1224/iptv_ipv4_live/refs/heads/main/live.m3u,央视频道,CCTV1,http://[2409:8087:1a01:df::7005]/ottrrs.hl.chinamobile.com/PLTV/88888888/224/3221226016/index.m3u8,IPV6,CCTV-1,央视频道,1.0,1.0,1,1046.014558
3804,https://raw.githubusercontent.com/kakaxi-1/IPTV/refs/heads/main/ipv6.m3u,央视频道,CCTV1,http://[2409:8087:1a15:10d1::32]/ottrrs.hl.chinamobile.com/PLTV/88888888/224/3221226016/index.m3u8,IPV6,CCTV-1,央视频道,1.0,1.0,1,1041.722175
4,https://raw.githubusercontent.com/Guovin/iptv-api/gd/output/result.m3u,📺央视频道,CCTV1,https://smart.pendy.dpdns.org/Smart.php?id=cctv1,IPV4,CCTV-1,央视频道,1.0,1.0,1,0.410655
1903,https://raw.githubusercontent.com/TianmuTNT/iptv/refs/heads/main/iptv.m3u,NaN,CCTV1,http://58.57.40.22:9901/tsfile/live/0001_1.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,0.000000
1904,https://raw.githubusercontent.com/TianmuTNT/iptv/refs/heads/main/iptv.m3u,NaN,CCTV1,https://cc06.tv12.xyz/1.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,0.000000
1905,https://raw.githubusercontent.com/TianmuTNT/iptv/refs/heads/main/iptv.m3u,NaN,CCTV1,http://play.kankanlive.com/live/1661761962676984.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,0.000000


In [11]:
data_filter_sort_head10=data_filter_sort.groupby('清洗频道名称').head(10)

In [12]:
data_filter_sort_head10.head(50)

,来源,频道组,频道名称,频道地址,频道类型,清洗频道名称,清洗频道组名称,频道组排序,频道排序,地址是否可用,测试速度
4224,https://raw.githubusercontent.com/alantang1977/iptv_api/refs/heads/main/output/live_ipv4.m3u,🥝┃央视频道,CCTV1,http://112.27.235.94:8000/hls/1/index.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,8915.056559
5,https://raw.githubusercontent.com/Guovin/iptv-api/gd/output/result.m3u,📺央视频道,CCTV1,http://[2409:8087:1:20:20::29]/otttv.bj.chinamobile.com/PLTV/88888888/224/3221226895/1.m3u8?serv...,IPV6,CCTV-1,央视频道,1.0,1.0,1,4183.204988
1,https://raw.githubusercontent.com/Guovin/iptv-api/gd/output/result.m3u,📺央视频道,CCTV1,http://php.jdshipin.com/TVOD/iptv.php?id=rthk33,IPV4,CCTV-1,央视频道,1.0,1.0,1,2882.702624
2,https://raw.githubusercontent.com/Guovin/iptv-api/gd/output/result.m3u,📺央视频道,CCTV1,http://iptv.huuc.edu.cn/hls/cctv1hd.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,1514.715191
6293,https://raw.githubusercontent.com/jiangnan1224/iptv_ipv4_live/refs/heads/main/live.m3u,央视频道,CCTV1,http://[2409:8087:1a01:df::7005]/ottrrs.hl.chinamobile.com/PLTV/88888888/224/3221226016/index.m3u8,IPV6,CCTV-1,央视频道,1.0,1.0,1,1046.014558
3804,https://raw.githubusercontent.com/kakaxi-1/IPTV/refs/heads/main/ipv6.m3u,央视频道,CCTV1,http://[2409:8087:1a15:10d1::32]/ottrrs.hl.chinamobile.com/PLTV/88888888/224/3221226016/index.m3u8,IPV6,CCTV-1,央视频道,1.0,1.0,1,1041.722175
4,https://raw.githubusercontent.com/Guovin/iptv-api/gd/output/result.m3u,📺央视频道,CCTV1,https://smart.pendy.dpdns.org/Smart.php?id=cctv1,IPV4,CCTV-1,央视频道,1.0,1.0,1,0.410655
1903,https://raw.githubusercontent.com/TianmuTNT/iptv/refs/heads/main/iptv.m3u,NaN,CCTV1,http://58.57.40.22:9901/tsfile/live/0001_1.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,0.000000
1904,https://raw.githubusercontent.com/TianmuTNT/iptv/refs/heads/main/iptv.m3u,NaN,CCTV1,https://cc06.tv12.xyz/1.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,0.000000
1905,https://raw.githubusercontent.com/TianmuTNT/iptv/refs/heads/main/iptv.m3u,NaN,CCTV1,http://play.kankanlive.com/live/1661761962676984.m3u8,IPV4,CCTV-1,央视频道,1.0,1.0,1,0.000000
